# Intro to theano.scan
In order to implement recurrent networks, we will need to be able to apply certain operations repeatedly. This can be handled by a for loop generating a theano expression, but if we do not know in advance how long the loop will be, this is not possible. In these situations we can use theano.scan to create these loops.

In [1]:
# imports we will need
import numpy as np
import theano
import theano.tensor as T
import numbers

Using gpu device 0: GRID K520


## Some simple examples
We begin by some very simple examples that should be coded as functions implementing for-loops

### Cumulative sum of integers
Write a function that returns the sum of the first `n` integers. Add a keyword `return_full_sequence=False`, which, if set to `True`, returns the full calculation sequence.

In [2]:
def cumulative_sum_int(n, return_full_sequence=False):
    result = 0
    if return_full_sequence:
        seq = []
    for i in range(1, n + 1):
        result += i
        if return_full_sequence:
            seq.append(result)
    if return_full_sequence:
        return seq
    return result

In [3]:
cumulative_sum_int(9, return_full_sequence=True)

[1, 3, 6, 10, 15, 21, 28, 36, 45]

In [4]:
cumulative_sum_int(10, return_full_sequence=False)

55

### Cumulative sum of squares
Do the same thing for a cumulative sum of squares (you can copy and paste the function to write the cumulative sum of integers)

In [5]:
def cumulative_sum_squares(n, return_full_sequence=False):
    result = 0
    if return_full_sequence:
        seq = []
    for i in range(1, n + 1):
        result += i ** 2
        if return_full_sequence:
            seq.append(result)
    if return_full_sequence:
        return seq
    return result

In [6]:
cumulative_sum_squares(9, return_full_sequence=True)

[1, 5, 14, 30, 55, 91, 140, 204, 285]

In [7]:
cumulative_sum_squares(10, return_full_sequence=False)

385

### Fibonacci series
Now write a function calucating the nth fibonacci number, the zeroth and first being 0 and 1

In [8]:
def fib(n, return_full_sequence=False):
    if n == 0:
        if return_full_sequence:
            return [0]
        return 0
    if n == 1:
        if return_full_sequence:
            return [0, 1]
        return 1
    pre1, pre2 = 1, 0
    if return_full_sequence:
        seq = [0, 1]
    for _ in range(2, n):
        pre1, pre2 = pre1 + pre2, pre1
        if return_full_sequence:
            seq.append(pre1)
    if return_full_sequence:
        return seq
    return pre1
    
        

In [9]:
fib(10, return_full_sequence=True)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [10]:
fib(11)

55

## Generalizing a little
All of the above can be written as simple operations done in a loop.
Code a function that can take a binary (two inputs) function of `new_value` and `current_value` and returns the increment. `new_value` should be read from a sequence. Add an argument `initial_value` to be able to commence the iterations. Output the full sequence.

In [11]:
def incrementer(func, sequence, initial_value):
    results = [initial_value]
    for item in sequence:
        results.append(func(item, results[-1]))
    return results[1:]
    

Write a function that adds two values in order to re-create the first example

In [12]:
def f_add(new_item, curr_value):
    return new_item + curr_value


Apply `incrementer` to `f_add` and a sequence of values from 0 to 9 with an initial value of 0

In [13]:
incrementer(f_add, np.arange(10), 0)

[0, 1, 3, 6, 10, 15, 21, 28, 36, 45]

Now write a `f_add_square` function and apply `incrementer` to it.

In [14]:
def f_add_square(new_item, curr_value):
    return new_item ** 2 + curr_value

In [15]:
incrementer(f_add_square, np.arange(10), 0.)

[0.0, 1.0, 5.0, 14.0, 30.0, 55.0, 91.0, 140.0, 204.0, 285.0]

## Generalizing a little more
If you have tried to implement the fibonacci example this way, you will have noticed that it is not possible to do this easily without access to the last *two* values calculated (instead of the last one). We thus need to extend our `incrementer` function to be able to handle this case.

Rewrite `incrementer` such that it accepts an argument `taps=(-1,)` which indicates which previous values of the calculated results you would like to make available to your function. It should call the agglomeration function first with the sequence value and then with these extra results. For `taps=(-1,)` it should behave exactly like `incrementer`

In [16]:
def incrementer2(func, sequence, initial_values, taps=(-1,)):
    result = [iv for iv in initial_values]
    taps = np.array(taps)
    index = len(result)
    for item in sequence:
        func_args = [item] + [result[index + tap] for tap in taps]
        result.append(func(*func_args))
        index += 1
    return result

In [17]:
def fib(curr, last, before_last):
    return last + before_last

In [18]:
incrementer2(fib, np.arange(10), [0, 1], taps=(-1, -2))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]

## Simple examples using theano.scan
We will now use the same examples in `theano.scan`

In [19]:
n = T.iscalar()
sum_int_vals, sum_int_updates = theano.scan(f_add, T.arange(n).astype('float32'), 0.)
sum_squares_vals, sum_squares_updates = theano.scan(f_add_square, T.arange(n).astype('float32'), 0.)

f_sum_int_vals = theano.function([n], sum_int_vals)
f_sum_squares_vals = theano.function([n], sum_squares_vals)

/home/ubuntu/venv/src/theano/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [20]:
f_sum_int_vals(20)

array([   0.,    1.,    3.,    6.,   10.,   15.,   21.,   28.,   36.,
         45.,   55.,   66.,   78.,   91.,  105.,  120.,  136.,  153.,
        171.,  190.], dtype=float32)

In [21]:
f_sum_squares_vals(10)

array([   0.,    1.,    5.,   14.,   30.,   55.,   91.,  140.,  204.,  285.], dtype=float32)

In order to implement Fibonacci numbers this way, we need to inform `theano.scan` of the necessary taps to use. This is done by passing a dictionary to `output_vals`

In [22]:
fibonacci_expr, fibonacci_updates = theano.scan(fib, T.arange(n), dict(initial=T.constant(np.array([0., 1.]).astype(np.float32)), taps=[-2, -1]))

In [23]:
f_fibonacci = theano.function([n], fibonacci_expr)

In [24]:
f_fibonacci(10)

array([  1.,   2.,   3.,   5.,   8.,  13.,  21.,  34.,  55.,  89.], dtype=float32)